In [ ]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row


def insert_key(x):
    return (x[0][0:6],(x[0],x[1]))



import apache_beam as beam
import time
from apache_beam.runners import DataflowRunner

p = beam.Pipeline(options=options)

batches_with_keys = (
      p
      | 'Create produce' >> beam.Create([
          ('INSBI479746850', 'oFIDjzDZJ'),('INSBI752381817','MNpdRhAYv'),('INSBI479746850', 'dfkhjgjzDZJ')
          
      ])
      |beam.Map(insert_key)
      | 'Group into batches' >> beam.GroupIntoBatches(2)
      | beam.FlatMap(lambda x: x[1])
      | beam.Map(makeRow)
      | "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table-groupintobatches',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='id:STRING,name:STRING',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       ))






pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_24698/484795044.py:58: FutureWarning: GroupIntoBatches is experimental.
  | 'Group into batches' >> beam.GroupIntoBatches(2)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [4]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row


def insert_key(x):
    return (x[0][0:6],(x[0],x[1]))



import apache_beam as beam
import time
from apache_beam.runners import DataflowRunner

p = beam.Pipeline(options=options)
    
batches_with_keys = (
      p
      | 'Create produce' >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/customer.csv')
      | beam.Map(lambda x: x.split(','))
      | beam.Map(insert_key)
      | 'Group into batches' >> beam.GroupIntoBatches(2)
      | beam.FlatMap(lambda x: x[1])
      | beam.Map(makeRow)
      | "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table-groupintobatches',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='id:STRING,name:STRING',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       ))






pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_24698/2878042032.py:59: FutureWarning: GroupIntoBatches is experimental.
  | 'Group into batches' >> beam.GroupIntoBatches(2)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [14]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)







from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

read_requests = p |beam.io.ReadFromBigQuery(method = beam.io.ReadFromBigQuery.Method.DIRECT_READ, table = 'mimetic-parity-378803.gcpdataset.my-table-groupintobatches')|beam.Map(print)
    
pipeline_result = DataflowRunner().run_pipeline(p, options=options)


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [15]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)







from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

with beam.Pipeline(options=options) as p:
    read_requests = p |beam.io.ReadFromBigQuery(method = beam.io.ReadFromBigQuery.Method.DIRECT_READ, table = 'mimetic-parity-378803.gcpdataset.my-table-groupintobatches')|beam.Map(print)
    
#pipeline_result = DataflowRunner().run_pipeline(p, options=options)

{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI752381817', 'name': 'MNpdRhAYv'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI752381817', 'name': 'MNpdRhAYv'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI752381817', 'name': 'MNpdRhAYv'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI752381817', 'name': 'MNpdRhAYv'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': '

In [25]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)







from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

with beam.Pipeline(options=options) as p:
    read_requests = p |beam.io.ReadFromBigQuery(query='SELECT id,name FROM '\
                                                      '`mimetic-parity-378803.gcpdataset.my-table-groupintobatches`',use_standard_sql=True )|beam.Map(print)
    
#pipeline_result = DataflowRunner().run_pipeline(p, options=options)

{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'INSBI082559854', 'name': 'GYsOJOnUx'}
{'id': 'INSBI084610576', 'name': 'NneVFZXsx'}
{'id': 'INSBI343728815', 'name': 'vLxlShTWA'}
{'id': 'INSBI394395590', 'name': 'zGIadoKbc'}
{'id': 'INSBI395828543', 'name': 'bVheDYhmY'}
{'id': 'INSBI475463658', 'name': 'PgrPXbNVZ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'oFIDjzDZJ'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': 'INSBI479746850', 'name': 'dfkhjgjzDZJ'}
{'id': '

In [28]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-groupintobatches` LIMIT 10 '




from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

with beam.Pipeline(options=options) as p:
    read_requests = p |beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )|beam.Map(print)
    
#pipeline_result = DataflowRunner().run_pipeline(p, options=options)

{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}
{'id': 'I', 'name': 'N'}


In [29]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 100'




from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

with beam.Pipeline(options=options) as p:
    read_requests = p |beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )|beam.Map(print)
    
#pipeline_result = DataflowRunner().run_pipeline(p, options=options)

{'id': 'INSBI481260153', 'name': 'ScQZUIHEv'}
{'id': 'INSBI074697300', 'name': 'uDvlVYENb'}
{'id': 'INSBI660679213', 'name': 'pCUaKPsft'}
{'id': 'INSBI803813880', 'name': 'mdlhybCQr'}
{'id': 'INSBI739035500', 'name': 'GgJVddxnV'}
{'id': 'INSBI191620337', 'name': 'boUCWVFMS'}
{'id': 'INSBI968522923', 'name': 'TrFhgvTND'}
{'id': 'INSBI604001572', 'name': 'srLQveGiV'}
{'id': 'INSBI998050361', 'name': 'GVXKacEMA'}
{'id': 'INSBI759226449', 'name': 'hthEUEqmx'}
{'id': 'INSBI850426845', 'name': 'frCuJshWe'}
{'id': 'INSBI987024042', 'name': 'VLhUkxYvg'}
{'id': 'INSBI047836722', 'name': 'ildcnseHp'}
{'id': 'INSBI427255631', 'name': 'UrRYCWqTh'}
{'id': 'INSBI603581665', 'name': 'ObCcEzUBE'}
{'id': 'INSBI857497968', 'name': 'zVOAZXzlb'}
{'id': 'INSBI371722292', 'name': 'kwgKTXzkw'}
{'id': 'INSBI187131681', 'name': 'mpdNVnwBw'}
{'id': 'INSBI002622428', 'name': 'bXWlKqOVD'}
{'id': 'INSBI258131620', 'name': 'QYoSKeHFG'}
{'id': 'INSBI152954662', 'name': 'EmPyQKCbo'}
{'id': 'INSBI652587880', 'name': '

In [44]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 100'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

#p = beam.Pipeline(options=options)

with beam.Pipeline(options=options) as p:
    read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |beam.Map(print))
    
#pipeline_result = DataflowRunner().run_pipeline(p, options=options)

('INSBI6', ('INSBI660679213', 'pCUaKPsft'))
('INSBI8', ('INSBI803813880', 'mdlhybCQr'))
('INSBI9', ('INSBI987024042', 'VLhUkxYvg'))
('INSBI0', ('INSBI047836722', 'ildcnseHp'))
('INSBI4', ('INSBI427255631', 'UrRYCWqTh'))
('INSBI6', ('INSBI603581665', 'ObCcEzUBE'))
('INSBI8', ('INSBI857497968', 'zVOAZXzlb'))
('INSBI9', ('INSBI998050361', 'GVXKacEMA'))
('INSBI6', ('INSBI631687786', 'eDBLgwZGn'))
('INSBI2', ('INSBI215614899', 'YdfUzErYL'))
('INSBI1', ('INSBI150212104', 'QKpwjymCn'))
('INSBI3', ('INSBI326594165', 'pFAAhdIfv'))
('INSBI1', ('INSBI130852732', 'kyWQuyrWh'))
('INSBI4', ('INSBI481260153', 'ScQZUIHEv'))
('INSBI0', ('INSBI074697300', 'uDvlVYENb'))
('INSBI4', ('INSBI458763878', 'HNLifNxpr'))
('INSBI3', ('INSBI366124244', 'TFrLULKOf'))
('INSBI5', ('INSBI526295416', 'LdvhimaIM'))
('INSBI2', ('INSBI249849907', 'OwRjEhjAW'))
('INSBI7', ('INSBI739035500', 'GgJVddxnV'))
('INSBI1', ('INSBI191620337', 'boUCWVFMS'))
('INSBI9', ('INSBI968522923', 'TrFhgvTND'))
('INSBI6', ('INSBI604001572', 's

In [47]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult


#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(50000)
    |beam.FlatMap(lambda x: x[1])
    |beam.Map(makeRow)
    |"Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table-output',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='id:STRING,name:STRING',
                                                                       create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       ))
    #|beam.Map(print))
    
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_24698/3802925769.py:59: FutureWarning: GroupIntoBatches is experimental.
  |'Group into batches' >> beam.GroupIntoBatches(50000)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [53]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import WorkerOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#number of workers
options.view_as(WorkerOptions).num_workers = 2

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)




query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(50000)
    |beam.FlatMap(lambda x: x[1])
    |beam.Map(makeRow)
    |"Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table-output',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='id:STRING,name:STRING',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       ))
    #|beam.Map(print))
    
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_24698/3089242466.py:64: FutureWarning: GroupIntoBatches is experimental.
  |'Group into batches' >> beam.GroupIntoBatches(50000)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [66]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import WorkerOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#number of workers
options.view_as(WorkerOptions).num_workers = 2

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)





query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

class make_api_call(beam.DoFn):
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        
        #content = urllib.request.urlopen('http://35.239.64.67:32726/').read().decode('utf-8')
        #row = dict(zip(('message'),content))
        
        
        
        # The API endpoint
        #url = "http://35.239.64.67:32726/"
        url = "https://jsonplaceholder.typicode.com/posts/1"

        # A GET request to the API
        response = requests.get(url)

        # Print the response
        response_json = response.json()


        #x = response_json['userId']
        #x1 = response_json['id']
        #x2 = response_json['title']
        #x3 = response_json['body']
        x=1
        x1=1
        x2='string'
        x3='strong'
        
        row = dict(zip(('userId','id','title','body'),(x,x1,x2,x3)))
        return row
          

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(50000)
    |'Make an External API Call' >> beam.ParDo(make_api_call())
    #|'Write To Text'>>beam.io.WriteToText('gs://gcp-dataeng-demos-soumya/dataflow/myfile.txt'))
    #|beam.Map(print))
               )
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_24698/2683731785.py:102: FutureWarning: GroupIntoBatches is experimental.
  |'Group into batches' >> beam.GroupIntoBatches(50000)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [74]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import WorkerOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#number of workers
options.view_as(WorkerOptions).num_workers = 2

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)





query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

class make_api_call(beam.DoFn):
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        #from kafka import KafkaProducer
        
        #producer = KafkaProducer(bootstrap_servers=['34.28.118.32:9094'], api_version=(0, 10))
        
        #content = urllib.request.urlopen('http://35.239.64.67:32726/').read().decode('utf-8')
        #row = dict(zip(('message'),content))
        
        
        
        # The API endpoint
        #url = "http://35.239.64.67:32726/"
        url = "https://jsonplaceholder.typicode.com/posts/1"

        # A GET request to the API
        response = requests.get(url)

        # Print the response
        response_json = response.json()
        #producer.send('my-topic', json.dumps(response_json).encode('utf-8'))
        #if producer is not None:
                    #producer.close()


        x = response_json['userId']
        x1 = response_json['id']
        x2 = response_json['title']
        x3 = response_json['body']
        #x=1
        #x1=1
        #x2='string'
        #x3='strong'
        
        row = dict(zip(('userId','id','title','body'),(x,x1,x2,x3)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator
          

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(50000)
    |'Make an External API Call' >> beam.ParDo(make_api_call())
    |"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery(        'my-table-message',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='userId:INTEGER, id:INTEGER,title:STRING,body:STRING',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    )
    #|beam.Map(print))
               
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_24698/716552436.py:107: FutureWarning: GroupIntoBatches is experimental.
  |'Group into batches' >> beam.GroupIntoBatches(50000)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [100]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import WorkerOptions
import apache_beam as beam
import time
from apache_beam.runners import DataflowRunner


options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#number of workers
options.view_as(WorkerOptions).num_workers = 2

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


########################################################################  With A Single Worker    ############################################################################


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        #from kafka import KafkaProducer
        
        #producer = KafkaProducer(bootstrap_servers=['34.28.118.32:9094'], api_version=(0, 10))
        
        #content = urllib.request.urlopen('http://35.239.64.67:32726/').read().decode('utf-8')
        #row = dict(zip(('message'),content))
        
        
        
        # The API endpoint
        #url = "http://35.239.64.67:32726/"
        url = "https://jsonplaceholder.typicode.com/posts/1"

        # A GET request to the API
        response = requests.get(url)

        # Print the response
        response_json = response.json()
        #producer.send('my-topic', json.dumps(response_json).encode('utf-8'))
        #if producer is not None:
                    #producer.close()

        length = len(element[1])
        x = response_json['userId']
        x1 = response_json['id']
        x2 = response_json['title']
        x3 = response_json['body']
        #x=1
        #x1=1
        #x2='string'
        #x3='strong'
        
        row = dict(zip(('userId','id','title','body','length'),(x,x1,x2,x3,length)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator
          

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(50000)
    |'Make an External API Call' >> beam.ParDo(make_api_call())
    |"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery(        'my-table-message-second',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='userId:INTEGER, id:INTEGER,title:STRING,body:STRING,length:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    )
    #|beam.Map(print))
               
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_24698/1199227284.py:108: FutureWarning: GroupIntoBatches is experimental.
  |'Group into batches' >> beam.GroupIntoBatches(50000)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [ ]:
https://cloud.google.com/eventarc/docs/gke/route-trigger-cloud-audit-logs

In [97]:
from platform import python_version
print(python_version())

3.8.10


In [98]:
import numpy
print(numpy.__version__)

1.22.4


In [99]:
import apache_beam
print(apache_beam.__version__)

2.44.0
